In [ ]:
import os
from statistics import mean
import time
from typing import Dict, List

from sir_utilities.date_time import string_to_datetime
from sqlalchemy import create_engine

from locast.candle.candle import Candle
from locast.candle.candle_utility import CandleUtility as cu
from locast.candle.dydx.dydx_candle_mapping import DydxV4CandleMapping
from locast.candle.exchange_candle_mapper import ExchangeCandleMapper
from locast.candle.dydx.dydx_resolution import DydxResolution
from locast.candle.exchange import Exchange
from locast.candle_storage.sql.sqlite_candle_storage import SqliteCandleStorage
from tests.helper.candle_mockery.mock_dydx_candle_dicts import (
    mock_dydx_candle_dict_batch,
)

end_months = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12",
]
results: Dict[str, List[float]] = {"storing": [], "retrieving": [], "deleting": []}

month = end_months[10]

cached_candles: List[Candle] = []


async def main(cached_candles: List[Candle]) -> None:
    eth = "ETH-USD"
    start_str = "2024-01-01T00:00:00+00:00"
    exchange = Exchange.DYDX_V4
    resolution = DydxResolution.ONE_MINUTE

    candles: List[Candle] = []

    n = 10
    for i in range(1, n):
        if len(cached_candles) == 0:
            end_str = f"2024-{month}-01T00:01:00+00:00"
            start = string_to_datetime(start_str)
            end = string_to_datetime(end_str)
            amount = cu.amount_of_candles_in_range(start, end, resolution)

            eth_dicts = mock_dydx_candle_dict_batch(
                Exchange.DYDX_V4,
                DydxResolution.ONE_MINUTE.notation,
                eth,
                start_str,
                end_str,
                batch_size=amount,
            )
            v4_mapper = ExchangeCandleMapper(DydxV4CandleMapping())

            cached_candles = v4_mapper.to_candles(eth_dicts)

        candles = cached_candles

        print(f"🏁 RUN #{i} 💥")
        print(f"Mocked Amount ({eth}): {len(candles)}.")

        engine = create_engine("sqlite:///locast.db", echo=False)
        candle_storage = SqliteCandleStorage(engine)

        start_time = time.time()
        await candle_storage.store_candles(candles)
        duration = round(time.time() - start_time, 2)
        results["storing"].append(duration)

        print(f"Time to store {len(candles)} candles: ({duration} seconds).")

        start_time = time.time()
        retrieved_candles = await candle_storage.retrieve_cluster(
            exchange,
            eth,
            resolution,
        )
        duration = round(time.time() - start_time, 2)
        results["retrieving"].append(duration)
        print(
            f"Time to retrieve {len(retrieved_candles)} candles: ({duration} seconds)."
        )
        if len(retrieved_candles) > 0:
            print(
                f"Candles retrieved from {retrieved_candles[-1].started_at}, to: {retrieved_candles[0].started_at}."
            )
        else:
            print(f"Candles: {retrieved_candles}")

        start_time = time.time()
        cluster_info = await candle_storage.get_cluster_info(exchange, eth, resolution)
        duration = round(time.time() - start_time, 2)
        print(f"Time to get cluster info: ({duration} seconds).")

        if cluster_info.head and cluster_info.tail:
            print(f"Tail started at: {cluster_info.tail.started_at}.")
            print(f"Head started at: {cluster_info.head.started_at}.")

        start_time = time.time()
        await candle_storage.delete_cluster(exchange, eth, resolution)
        duration = round(time.time() - start_time, 2)
        results["deleting"].append(duration)

        print(f"Time to delete cluster: ({duration} seconds).")

        cluster_info = await candle_storage.get_cluster_info(exchange, eth, resolution)
        print(f"Cluster info: {cluster_info}.")

        print("❌ Deleting db file...")

        os.remove("locast.db")

    print(f"🏁 Final Results for {month} Month(s) 🏁")
    print(
        f"📀 Storing took {round(mean(results['storing']),2)} seconds on average (best: {min(results['storing'])} seconds, worst: {max(results['storing'])} seconds)."
    )

    print(
        f"🚛 Retrieving took {round(mean(results['retrieving']),2)} seconds on average (best: {min(results['retrieving'])} seconds, worst: {max(results['retrieving'])} seconds)."
    )

    print(
        f"🗑️ Deleting took {round(mean(results['deleting']),2)} seconds on average (best: {min(results['deleting'])} seconds, worst: {max(results['deleting'])} seconds)."
    )

    cached_candles = []


await main(cached_candles)